In [1]:
!pip install bitsandbytes

In [2]:
!pip install -q accelerate==0.21.0 

In [3]:
!pip install -q peft==0.4.0

In [4]:
!pip install -q torch

In [5]:
!pip install -q transformers==4.38.2

In [6]:
!pip install -q trl==0.4.7

**Accelerate** is a library that enables the same PyTorch code to be run across any distributed configuration by adding just four lines of code!

**PEFT** (Parameter-Efficient Fine-Tuning) is a library for efficiently adapting large pretrained models to various downstream applications without fine-tuning all of a model’s parameters because it is prohibitively costly. PEFT methods only fine-tune a small number of (extra) model parameters - significantly decreasing computational and storage costs - while yielding performance comparable to a fully fine-tuned model.

**bitsandbytes** enables accessible large language models via k-bit quantization for PyTorch. bitsandbytes provides three main features for dramatically reducing memory consumption for inference and training

**TRL** is a full stack library where we provide a set of tools to train transformer language models with Reinforcement Learning, from the Supervised Fine-tuning step (SFT), Reward Modeling step (RM) to the Proximal Policy Optimization (PPO) step. The library is integrated with 🤗 transformers.


## Step: Import all the required libraries

In [10]:
import os
import torch 
import accelerate
from datasets import load_dataset
from transformers import(
    AutoModelForCausalLM,
    AutoTokenizer, 
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

## In case of LLAMA 2, the following prompt template is used for the chat models

<s.> [INST] <<SYS.>>

System Prompt

<</SYS.>>

User Prompt [/INST] Model answer </s.>

## We Will reformat our dataset into the LLAMA model format

- Orignal Dataset: https://huggingface.co/datasets/timdettmers/openassistant-guanaco
  
- Reformat Dataset following the Llama 2 template with 1k sample: https://huggingface.co/datasets/mlabonne/guanaco-llama2-1k

- Complete Reformat Dataset following the Llama 2 template: https://huggingface.co/datasets/mlabonne/guanaco-llama2
  

To know how this dataset was created, you can check this notebook.

https://colab.research.google.com/drive/1Ad7a9zMmkxuXTOh1Z7-rNSICA4dybpM2?usp=sharing

### Note: You don’t need to follow a specific prompt template if you’re using the base Llama 2 model instead of the chat version.

## How to fine tune LLAMA 2

- Free Resources are barely enough to store LLAMA 2 - 7b weights
- We also need to consider the overhead due to optimizer states, gradients, and forward activations
- Full fine-tuning is not possible here: we need parameter-efficient fnue-tuning (PEFT) techniques like LoRA.
- To drastically reduce the VRAM usage, we must fine-tune the model in 4-bit precision, which is why we use LoRA here.

## Step 3

1. Load a LLAMA-2-7b-chat-hf model (chat model)
2. Train it on the mlabonne/guanaco-llama2-1k (1000samples), which will produce our fine-tuned model LLAMA-2-7b-chat-finetune

QLoRA will use a rank of 64 with a scaling parameter of 16. We'll load the Llama 2 model directly in 4-bit precision using the NF4 type and train it for one epoch. 

In [11]:
# The model that we are training from the hugging face hub
model_name = "NousResearch/Llama-2-7b-chat-hf"

# The Instruction dataset to use
dataset_name = "mlabonne/guanaco-llama2-1k"

# Fine-tuned model name
new_model = "Llama-2-7b-chat-finetune"

############################################################################################
# QLoRA parameters
############################################################################################

# LoRA attention dimension
lora_r = 64

# Aplha parameter for LoRA scaling
lora_alpha = 16 

#Dropout probability for LoRA layers 
lora_dropout = 0.1

############################################################################################
# bitsandbytes parameters 
############################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_type = "float16"

# Quantization Type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

############################################################################################
# TrainingArguments Parameters
############################################################################################


# Output directory where the model predictions and checkpoints will be stored 
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training 
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of updates steps to accumulate the gradients for 
gradient_accumulation_steps = 1

# Enable gradient Checkpointing 
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial Learning Rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights 
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for linear warmup (from 0 to learning rate)
warup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps 
save_steps = 0

# log every X updates steps
logging_steps = 25

############################################################################################
# SFT parameters
############################################################################################

# Maximum sequence length to use
max_seq_length = None 

# pack multiple short example in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"":0}

## Step 4 : Load Everything and start the fine tuning process

1. First we need to load the dataset we defined. In this case, our data is already preprocessed, we reformat the prompt, filter out bad text, combine multiple datasets, etc.

2. Then, we're configuring bitsandbytes for 4-bit quantization

3. Next, loading the Llama 2 model in 4-bit precision on a GPU with the corresponding tokenizer.

4. Finally, loading the configuration for QLoRA, regular training parameters, and pass everything to the SFTTrainer. Now, training can finally start!

In [16]:
!pip install --upgrade accelerate bitsandbytes


In [17]:
# Load dataset (you can process it here)
dataset = load_dataset(dataset_name, split = "train")

# load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_type)

bnb_config = BitsAndBytesConfig(
    load_in_4bit = use_4bit, 
    bnb_4bit_quant_type = bnb_4bit_quant_type, 
    bnb_4bit_compute_dtype = compute_dtype,
    bnb_4bit_use_double_quant = use_nested_quant,
)

# # Check GPU compatibility with bfloat16
# if compute_dtype ==torch.float16 and use_4bit:
#     major, _ = torch.cuda.get_device_capability()
#     if major >= 8:
#         print("=" * 80)
#         print("Your GPU supports bfloat16 : accelerate training with bf16=True")
#         print("=" * 80)


print(accelerate.__version__)
# Load base model 
model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    quantization_config = bnb_config,
    device_map = device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMa tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code = True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha = lora_alpha,
    lora_dropout = lora_dropout,
    r = lora_r,
    bias = "none",
    task_type = "CAUSAL_LM",
)

# Set training parameters 
training_arguments = TrainingArguments(
    output_dir = output_dir, 
    num_train_epochs = num_train_epochs, 
    per_device_train_batch_size = per_device_train_batch_size, 
    gradient_accumulation_steps = gradient_accumulation_steps,
    optim=optim, 
    save_steps = save_steps,
    logging_steps = logging_steps, 
    learning_rate = learning_rate, 
    weight_decay = weight_decay,
    fp16 = fp16, 
    bf16 = bf16, 
    max_grad_norm = max_grad_norm, 
    max_steps = max_steps, 
    warmup_ratio = warmup_ratio, 
    group_by_length = group_by_length, 
    lr_scheduler_type = lr_scheduler_type, 
    report_to = "tensorboard"
)

# Set supervised fine-tuning parameters 
trainer = SFTTrainer(
    model = model, 
    train_dataset = dataset, 
    peft_config = peft_config, 
    dataset_text_field = "text", 
    max_seq_length = max_seq_length, 
    tokenizer = tokenizer, 
    args = training_arguments, 
    packing = packing,
)

# Train Model 
trainer.train()

0.21.0


ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`